In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-18 18:45:34,888 - NumExpr defaulting to 8 threads.


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2050425176128) was called 1 times


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2050425178144) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2050425177856) was called 1 times


In [ ]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2050425178720) was called 1 times
Running: Function "load_module_lookup" (id=2050425179440) was called 1 times


In [ ]:
fa_floor = feat.floor_shift_time_fa(fa, shift=15)

In [ ]:
fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Both','Blue','Grey']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'Asset Code', agg_type = 'count')

In [ ]:
av_sel,at_sel = feat.av_at_select(av, at, availability_select_options = None, remove_high_AT = True, AT_limit = None)

av_agg = feat.aggregate_availability(av_sel, agg_level ='None')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'None')

#at_agg = at_agg[at_agg['TOTES']>100]

In [ ]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg , agg_level = 'None')

In [ ]:
X,y = md.gen_feat_var(df,target = 'Blue Tote Loss', features = ['Totes'])
y=y

In [ ]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'threshold': -np.inf,
                    'max_features': 200}

X_sel_rf = md.select_features(X, y, RF_mdl, **selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, **selection_options)

#X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,split_options = {'test_size': 0.3,
                                                                       'random_state': None})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X, y)

### Add LM - metrics

In [ ]:
sc.lm_coefficients(Linear_mdl, X_test ,y_test)

In [ ]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

In [ ]:
df_greyhigh_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_ID.sum().sort_values(ascending=False).head(20)

In [ ]:
df_grey_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_ID.sum().sort_values(ascending=False).head(20)

In [ ]:
df_greyhigh_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_Asset.sum().sort_values(ascending=False).head(20)

In [ ]:
df_grey_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_Asset.sum().sort_values(ascending=False).head(20)

In [ ]:
av_agg

In [ ]:
y